In [84]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import geocoder
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [85]:
#pip install geocoder

In [86]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

In [98]:
url = 'https://en.wikipedia.org/wiki/Category:Suburbs_of_Sydney'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.findAll('div', {'class': "mw-category-group"})



In [99]:
suburbs = []
for tag in soup.find_all("li"):
    if(', New South Wales' in tag.text):
        text = tag.text
        i = 0
        while(not text[i].isalpha()):
            i = i + 1        
        
        suburbs.append(tag.text[i:tag.text.index(", New South Wales")+10])

len(suburbs)

329

In [89]:
# create a new DataFrame from the list
syd_sub = pd.DataFrame({"Suburbs": suburbs})
syd_sub.head()

,Suburbs
0,"Alexandria, New Sout"
1,"Annandale, New Sout"
2,"Arncliffe, New Sout"
3,"Artarmon, New Sout"
4,"Ashfield, New Sout"


## Retrieving Geographical Coordinates

In [90]:
# code for getting the latitude and longitude
def get_lati_long(suburb):
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Sydney, New South Wales'.format(suburb))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [91]:
suburb_names = syd_sub['Suburbs'].tolist()

In [92]:
lat_lang = [get_lati_long(suburb) for suburb in suburb_names]

In [93]:
df_coords = pd.DataFrame(lat_lang, columns=['Latitude', 'Longitude'])
syd_sub['Latitude'] = df_coords['Latitude']
syd_sub['Longitude'] = df_coords['Longitude']

In [94]:
syd_sub.head()

,Suburbs,Latitude,Longitude
0,"Alexandria, New Sout",-33.91237,151.19703
1,"Annandale, New Sout",-33.88005,151.17130
2,"Arncliffe, New Sout",-33.93665,151.14679
3,"Artarmon, New Sout",-33.80814,151.18380
4,"Ashfield, New Sout",-33.88920,151.12543


In [95]:
syd_sub.to_csv('sydney.csv',index=False)

## Getting Location Data of Sydney

In [81]:
syd_data = pd.read_csv('sydney.csv')
syd_data.head()

,Suburbs,Latitude,Longitude
0,"Alexandria, New Sout",-33.91237,151.19703
1,"Annandale, New Sout",-33.88005,151.17130
2,"Arncliffe, New Sout",-33.93665,151.14679
3,"Artarmon, New Sout",-33.80814,151.18380
4,"Ashfield, New Sout",-33.88920,151.12543


In [82]:
address = 'Sydney, New South Wales, Australia'

geolocator = Nominatim(user_agent="sydney_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney, Australia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sydney, Australia are -33.8548157, 151.2164539.


In [83]:
neighborhood_latitude = syd_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = syd_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = syd_data.loc[0, 'Suburbs'] # neighborhood name

In [30]:
# create map of Manhattan using latitude and longitude values
map_Syd = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, label in zip(syd_data['Latitude'], syd_data['Longitude'], syd_data['Suburbs']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Syd)  
    
map_Syd

In [34]:
CLIENT_ID = 'BJC3T1QUFD5INSCA0K51KAXDCMBC3YISZBYOX0CPWBWT5GWC' # your Foursquare ID
CLIENT_SECRET = 'BLZVGUCOYFFLYXR4FSNWJKZIQWXHHNEFXFO2IWLGTCSR03UL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BJC3T1QUFD5INSCA0K51KAXDCMBC3YISZBYOX0CPWBWT5GWC
CLIENT_SECRET:BLZVGUCOYFFLYXR4FSNWJKZIQWXHHNEFXFO2IWLGTCSR03UL


In [35]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
# Part 1: creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
# Part 2: making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
    
    
    
        
# Part 3 returning only relevant information for each nearby venue and append to the list 
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    return (venues_list)

In [36]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=BJC3T1QUFD5INSCA0K51KAXDCMBC3YISZBYOX0CPWBWT5GWC&client_secret=BLZVGUCOYFFLYXR4FSNWJKZIQWXHHNEFXFO2IWLGTCSR03UL&v=20180605&ll=-33.912369999999946,151.19703000000004&radius=500&limit=100'

In [37]:
results = requests.get(url).json()

### Get all the Nearby Venues in Sydney

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburbs', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# type your answer here
syd_venues = getNearbyVenues(names=syd_data['Suburbs'], latitudes=syd_data['Latitude'], longitudes=syd_data['Longitude'])

Alexandria, New Sout
Annandale, New Sout
Arncliffe, New Sout
Artarmon, New Sout
Ashfield, New Sout
Asquith, New Sout
Auburn, New Sout
Audley, New Sout
Avalon Beach, New Sout
Balmain East, New Sout
Balmain, New Sout
Bankstown, New Sout
Bargo, New Sout
Baulkham Hills, New Sout
Bella Vista, New Sout
Bellevue Hill, New Sout
Belrose, New Sout
Birchgrove, New Sout
Bondi Beach, New Sout
Bondi Junction, New Sout
Bondi, New Sout
Botany, New Sout
Bronte, New Sout
Brooklyn, New Sout
Burwood, New Sout
Cabramatta, New Sout
Camellia, New Sout
Campbelltown, New Sout
Camperdown, New Sout
Canterbury, New Sout
Caringbah South, New Sout
Castle Hill, New Sout
Castlecrag, New Sout
Centennial Park, New Sout
Chatswood West, New Sout
Chatswood, New Sout
Chester Hill, New Sout
Chippendale, New Sout
Concord, New Sout
Coogee, New Sout
Cremorne, New Sout
Cronulla, New Sout
Crows Nest, New Sout
Croydon, New Sout
Darling Point, New Sout
Darlinghurst, New Sout
Darlington, New Sout
Dawes Point, New Sout
Denistone, Ne

## Analyse the Dataset

In [41]:
# one hot encoding
syd_onehot = pd.get_dummies(syd_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
syd_onehot['Suburbs'] = syd_venues['Suburbs'] 

# move neighborhood column to the first column
fixed_columns = [syd_onehot.columns[-1]] + list(syd_onehot.columns[:-1])
syd_onehot = syd_onehot[fixed_columns]

syd_onehot.head()

,Suburbs,Accessories Store,Adult Boutique,Airport,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Warehouse Store,Water Park,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio
0,"Alexandria, New Sout",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alexandria, New Sout",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Alexandria, New Sout",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Alexandria, New Sout",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Alexandria, New Sout",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
#get_dummies method of Pandas
syd_grouped = syd_onehot.groupby('Suburbs').mean().reset_index()
syd_grouped

,Suburbs,Accessories Store,Adult Boutique,Airport,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Warehouse Store,Water Park,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio
0,"Abbotsbury, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,"Abbotsford, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0
2,"Acacia Gardens, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,"Airds, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,"Alexandria, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,"Wilberforce, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
259,"Willoughby, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
260,"Windsor, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
261,"Woollahra, New Sout",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012048,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburbs']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Suburbs'] = syd_grouped['Suburbs']

for ind in np.arange(syd_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(syd_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Suburbs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abbotsbury, New Sout",Supermarket,Park,Convenience Store,Bus Stop,Yoga Studio,Flower Shop,Film Studio,Fish & Chips Shop,Fish Market,Fishing Spot
1,"Abbotsford, New Sout",Café,Park,Wine Shop,Thai Restaurant,Pizza Place,Fishing Store,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market
2,"Acacia Gardens, New Sout",Park,BBQ Joint,Yoga Studio,Flower Shop,Film Studio,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market
3,"Airds, New Sout",Pub,Candy Store,Shopping Mall,Yoga Studio,Fishing Store,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market
4,"Alexandria, New Sout",Café,Electronics Store,Flea Market,Grocery Store,Liquor Store,Lebanese Restaurant,Basketball Stadium,Coffee Shop,Sandwich Place,Bar


In [49]:
neighborhoods_venues_sorted.describe()

,Suburbs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,263,263,263,263,263,263,263,263,263,263,263
unique,263,70,85,97,85,84,82,87,85,74,77
top,"Chifley, New Sout",Café,Park,Yoga Studio,Yoga Studio,Yoga Studio,Film Studio,Film Studio,Fish Market,Fishing Spot,Fishing Spot
freq,1,102,19,19,18,20,29,31,31,31,32


In [50]:
res_syd = syd_grouped[["Suburbs", "Café"]]

In [51]:
res_syd

,Suburbs,Café
0,"Abbotsbury, New Sout",0.000000
1,"Abbotsford, New Sout",0.333333
2,"Acacia Gardens, New Sout",0.000000
3,"Airds, New Sout",0.000000
4,"Alexandria, New Sout",0.280000
...,...,...
258,"Wilberforce, New Sout",0.000000
259,"Willoughby, New Sout",0.250000
260,"Windsor, New Sout",0.173913
261,"Woollahra, New Sout",0.096386



## Cluster Neighborhoods

In [53]:
# set number of clusters
kclusters = 3

syd_grouped_clustering = res_syd.drop('Suburbs', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(syd_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 2, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 2, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 2, 1, 0, 2, 1, 1, 2, 2, 2, 2, 0, 2, 1,
       1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 0, 2, 1,
       0, 1, 1, 0, 2, 0, 1, 0, 0, 2, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 2,
       1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 2, 0, 0, 1,
       1, 1, 0, 1, 1, 2, 2, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 2, 1, 1, 1,
       2, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 2, 1, 0, 0, 0, 2,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 2, 2, 1,
       2, 1, 0, 0, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0])

In [54]:
syd_data

,Suburbs,Latitude,Longitude
0,"Alexandria, New Sout",-33.912370,151.197030
1,"Annandale, New Sout",-33.880050,151.171300
2,"Arncliffe, New Sout",-33.936650,151.146790
3,"Artarmon, New Sout",-33.808140,151.183800
4,"Ashfield, New Sout",-33.889200,151.125430
...,...,...,...
324,"Dawes Point, New Sout",-33.855510,151.208250
325,"Dean Park, New Sout",-33.735160,150.852280
326,"Denham Court, New Sout",-33.981427,150.832667
327,"Denistone East, New Sout",-33.794380,151.097840


In [56]:
# add clustering labels
res_syd.insert(0, 'Cluster Labels', kmeans.labels_)

In [61]:
syd_merged = syd_data[syd_data['Suburbs'].isin(res_syd.Suburbs)].reset_index()

In [62]:
# merge melb_grouped with melb_data to add latitude/longitude for each neighborhood
syd_merged = syd_merged.join(res_syd.set_index('Suburbs'), on='Suburbs')

syd_merged["Cluster Labels"].dropna(inplace = True)

syd_merged

,index,Suburbs,Latitude,Longitude,Cluster Labels,Café
0,0,"Alexandria, New Sout",-33.91237,151.19703,0,0.280000
1,1,"Annandale, New Sout",-33.88005,151.17130,0,0.291667
2,2,"Arncliffe, New Sout",-33.93665,151.14679,0,0.100000
3,3,"Artarmon, New Sout",-33.80814,151.18380,0,0.200000
4,4,"Ashfield, New Sout",-33.88920,151.12543,1,0.081081
...,...,...,...,...,...,...
300,321,"Darlinghurst, New Sout",-33.87915,151.22021,0,0.180000
301,322,"Darlington, New Sout",-33.89185,151.19356,0,0.222222
302,324,"Dawes Point, New Sout",-33.85551,151.20825,0,0.140845
303,325,"Dean Park, New Sout",-33.73516,150.85228,1,0.000000


In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(syd_merged['Latitude'], syd_merged['Longitude'], syd_merged['Suburbs'], syd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

--> Examine the Clusters

## Cluster 1

In [66]:
syd_merged.loc[syd_merged['Cluster Labels'] == 0, syd_merged.columns[[1] + list(range(5, syd_merged.shape[1]))]]

,Suburbs,Café
0,"Alexandria, New Sout",0.280000
1,"Annandale, New Sout",0.291667
2,"Arncliffe, New Sout",0.100000
3,"Artarmon, New Sout",0.200000
7,"Audley, New Sout",0.189189
...,...,...
293,"Cronulla, New Sout",0.181818
294,"Crows Nest, New Sout",0.191489
300,"Darlinghurst, New Sout",0.180000
301,"Darlington, New Sout",0.222222


## Cluster 2 

In [67]:
syd_merged.loc[syd_merged['Cluster Labels'] == 1, syd_merged.columns[[1] + list(range(5, syd_merged.shape[1]))]]

,Suburbs,Café
4,"Ashfield, New Sout",0.081081
5,"Asquith, New Sout",0.000000
6,"Auburn, New Sout",0.000000
17,"Birchgrove, New Sout",0.000000
19,"Bondi Junction, New Sout",0.056604
...,...,...
296,"Croydon, New Sout",0.000000
297,"Cumberland Reach, New Sout",0.000000
298,"Currans Hill, New Sout",0.000000
303,"Dean Park, New Sout",0.000000


## Cluster 3 

In [68]:
syd_merged.loc[syd_merged['Cluster Labels'] == 1, syd_merged.columns[[1] + list(range(5, syd_merged.shape[1]))]]

,Suburbs,Café
4,"Ashfield, New Sout",0.081081
5,"Asquith, New Sout",0.000000
6,"Auburn, New Sout",0.000000
17,"Birchgrove, New Sout",0.000000
19,"Bondi Junction, New Sout",0.056604
...,...,...
296,"Croydon, New Sout",0.000000
297,"Cumberland Reach, New Sout",0.000000
298,"Currans Hill, New Sout",0.000000
303,"Dean Park, New Sout",0.000000
